## News :D



In [18]:
%load_ext autoreload
%autoreload 2

import requests
import re # imports regex
from bs4 import BeautifulSoup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Installs:
lxmn
bs4

In [38]:
months = ['January', 'February','March','April','May','June','July','August','September','October','November','December']

cnn_links = []
for i in range(2014, 2024):
    for j in range(1, 13):
        cnn_links.append(f'https://www.cnn.com/article/sitemap-{i}-{j}.html')
grouped_cnn_links = [[f'https://www.cnn.com/article/sitemap-{i}-{j}.html' for j in range(1,13)] for i in range(2014, 2024)]
# print('CNN Links: ' + str(cnn_links))


nyt_links = []
for i in range(2014, 2024):
    for j in range(1, 13):
        nyt_links.append(f'https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/')
grouped_nyt_links = [[f'https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/' for j in range(1,13)] for i in range(2014, 2024)]
# print('New York Times Links: ' + str(nyt_links))


bi_links = []
for i in range(2014, 2024):
    for j in range(1, 13):
        bi_links.append(f'https://www.businessinsider.com/sitemap/html/{i}-{j}.html')
grouped_bi_links = [[f'https://www.businessinsider.com/sitemap/html/{i}-{j}.html' for j in range(1,13)] for i in range(2014, 2024)]
# print('Business Insider Links: ' + str(bi_links))


th_links = []
for i in range(2014, 2024):
    for j in months:
        th_links.append(f'https://thehill.com/sitemap/{i}/{j}')
grouped_th_links = [[f'https://thehill.com/sitemap/{i}/{j}' for j in months] for i in range(2014, 2024)]
# print('The Hill Links: ' + str(th_links))


nyp_links = []
for i in range(2014, 2024):
    for j in range(1,13):
        nyp_links.append(f'https://nypost.com/{i}/{str(j).zfill(2)}/')
grouped_nyp_links = [[f'https://nypost.com/{i}/{str(j).zfill(2)}/' for j in range(1,13)] for i in range(2014, 2024)]
# print('New York Post Links: ' + str(nyp_links))


dm_links = []
for i in range(2014, 2024):
    for j in range(1,13):
        dm_links.append(f'https://www.dailymail.co.uk/home/sitemaparchive/month_{i}{str(j).zfill(2)}/.html')
grouped_dm_links = [[f'https://www.dailymail.co.uk/home/sitemaparchive/month_{i}{str(j).zfill(2)}/.html' for j in range(1,13)] for i in range(2014, 2024)]
# print('Daily Mail Links: ' + str(dm_links))


fox_links = ['https://www.foxnews.com/sitemap.xml?type=articles'] + [f'https://www.foxnews.com/sitemap.xml?type=articles&page={i}' for i in range(2, 161)]
# print(fox_links)